In [51]:
import pandas as pd
data_path = '../data/skylab_instagram_datathon_dataset.csv'

data = pd.read_csv(data_path, delimiter=';')  # Specify the delimiter (e.g., comma)

data["period_end_date"] = pd.to_datetime(data["period_end_date"])



In [52]:
important_cols = ["followers","pictures","videos","comments","likes"]
funnylist = []

for group_idx in data["business_entity_doing_business_as_name"].unique()[:10]:
    filtered_df = data[data["business_entity_doing_business_as_name"] == group_idx]
    filtered_sum = filtered_df.isna().sum()
    funnylist.append((group_idx,
                      sum(filtered_sum[important_cols])/len(filtered_df)))

funnydict = {idx:cumulative for idx,cumulative in funnylist}
funnydict

{'Versace': 0.0,
 "Tim Horton's": 0.06373626373626373,
 'Gatorade': 0.0,
 'Monster Energy': 0.3648351648351648,
 'Aritzia': 0.33002481389578164,
 'Lovisa': 0.4119106699751861,
 'Saks Fifth Avenue': 0.0,
 'Ferragamo': 0.013186813186813187,
 'Riachuelo': 0.03736263736263736,
 'Louis Vuitton': 0.0}

In [53]:
import matplotlib.pyplot as plt

def calculate_yearly_means(df, col, category_column, category_value, period: str="yearly"):
    filtered_df = df[df[category_column] == category_value]
    
    # Convert the index to datetime if it's not already
    if period == "yearly":
        filtered_df["time"] = pd.to_datetime(filtered_df["period_end_date"]).dt.year
    elif period == "montly":
        filtered_df["time"] = pd.to_datetime(filtered_df["period_end_date"]).dt.month

    yearly_means = filtered_df[col].groupby(filtered_df["time"]).mean()
    
    return yearly_means


metrics = ["followers", "pictures", "likes", "videos", "comments"]
'''
for col in metrics:
    compset_groups = data["compset_group"].unique()
    num_columns = len(compset_groups)
    fig, axes = plt.subplots(num_columns, 1, figsize=(10, 6*num_columns))
    plt.title(col)
    for i, cat in enumerate(compset_groups):
        cat_means = calculate_yearly_means(df=data, col="followers", category_column="compset", category_value=cat)
        cat_means.plot(ax=axes[i], title=cat)
        axes[i].set_ylabel('Mean' + ' ' + col.capitalize())
        axes[i].set_xlabel('Year')

plt.tight_layout()
plt.show()
'''

'\nfor col in metrics:\n    compset_groups = data["compset_group"].unique()\n    num_columns = len(compset_groups)\n    fig, axes = plt.subplots(num_columns, 1, figsize=(10, 6*num_columns))\n    plt.title(col)\n    for i, cat in enumerate(compset_groups):\n        cat_means = calculate_yearly_means(df=data, col="followers", category_column="compset", category_value=cat)\n        cat_means.plot(ax=axes[i], title=cat)\n        axes[i].set_ylabel(\'Mean\' + \' \' + col.capitalize())\n        axes[i].set_xlabel(\'Year\')\n\nplt.tight_layout()\nplt.show()\n'

In [83]:
# Assuming 'data' is your DataFrame and 'metrics' is a list of columns to interpolate
data = data.sort_values("period_end_date")

# Group by the company name and apply interpolation within each group
for company in data["business_entity_doing_business_as_name"].unique():
    company_idx = (data["business_entity_doing_business_as_name"] == company)
    filtered = data[company_idx]
    filtered[metrics] = filtered[metrics].interpolate(method="linear")
    data[company_idx] = filtered


doof = data[data["business_entity_doing_business_as_name"] == "Hering"]
doof = doof[metrics]

/tmp/ipykernel_14251/278987916.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered[metrics] = filtered[metrics].interpolate(method="linear")
/tmp/ipykernel_14251/278987916.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered[metrics] = filtered[metrics].interpolate(method="linear")
/tmp/ipykernel_14251/278987916.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [89]:
counter = 0 
no_companies = len(data["business_entity_doing_business_as_name"].unique())
for company in data["business_entity_doing_business_as_name"].unique():
    company_idx = (data["business_entity_doing_business_as_name"] == company)
    filtered = data[company_idx]
    if any(filtered[metrics].isna()):
        counter += 1

print(counter/no_companies)

1.0
